# 浪子之心科技lil
# ImageTalking-Wav2Lip-CodeFormer
# 此代码仅为演示输入一张图片，然后动作迁移将图片模仿成视频，再用wav2lip进行语言驱动嘴型，最后用CodeFormer进行高清化的过程。
# 效果比sadtalker好。
# 此方案本人再3月份已经开源到B站视频，现在将代码一起开源。
# 主要步骤一：输入一张图片，用DaGAN，DPE，Thin-Plate-Spline-Motion-Model，first-order-model，StyleHEAT等动作迁移算法将图片迁移成视频。
# 主要步骤二：输入音频，用wav2lip或者video-retalking,DInet等数字人驱动算法将迁移后的视频驱动成说话的视频。
# 主要步骤三：用CodeFormer，GFPGAN，PGEN等进行后期超分修复。


In [1]:
#@title 一、下载wav2lip源码和权重并安装wav2lip环境
#@title
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [2]:
#@title 二、下载CodeFormer的源码权重并安装依赖

# 下载CodeFormer的源码
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer
# 下载下载CodeFormer的权重
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

# 安装CodeFormer的依赖
%cd /content/CodeFormer
!python basicsr/setup.py develop

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 583, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 583 (delta 168), reused 166 (delta 144), pack-reused 349
Receiving objects: 100% (583/583), 17.31 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/CodeFormer
/content/CodeFormer
Traceback (most recent call last):
  File "/content/CodeFormer/scripts/download_pretrained_models.py", line 5, in <module>
    from basicsr.utils.download_util import load_file_from_url
ModuleNotFoundError: No module named 'basicsr'
Traceback (most recent call last):
  File "/content/CodeFormer/scripts/download_pretrained_models.py", line 5, in <module>
    from basicsr.utils.download_util import load_file_from_url
ModuleNotFoundError: No module named 'basicsr'
/content/CodeFormer
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_bu

In [3]:
#@title 三、下载TPSMM源码权重并根据需要按照依赖
# 下载Thin-Plate-Spline-Motion-Model源码
%cd /content
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model
# 下载权重
%cd /content/Thin-Plate-Spline-Motion-Model
!mkdir checkpoints
%cd /content/Thin-Plate-Spline-Motion-Model/checkpoints
!gdown https://drive.google.com/uc\?id\=1bkX6GKK5e5RyPdr6SiBw-FZekhO9Z7ZS
# 按照相关依赖
%cd /content/Thin-Plate-Spline-Motion-Model


/content
Cloning into 'Thin-Plate-Spline-Motion-Model'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 112 (delta 41), reused 29 (delta 29), pack-reused 50
Receiving objects: 100% (112/112), 32.65 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/Thin-Plate-Spline-Motion-Model
/content/Thin-Plate-Spline-Motion-Model/checkpoints
Downloading...
From: https://drive.google.com/uc?id=1bkX6GKK5e5RyPdr6SiBw-FZekhO9Z7ZS
To: /content/Thin-Plate-Spline-Motion-Model/checkpoints/vox.pth.tar
100% 351M/351M [00:07<00:00, 46.8MB/s]
/content/Thin-Plate-Spline-Motion-Model


In [4]:
#@title 四、上传音频
# 创建相关的文件夹
# 如果只是运行demo,点击取消上传即可
%cd /content
!rm -rf sample_data
!rm -rf input_audio
!mkdir output
!mkdir input_audio
%cd /content/input_audio
!gdown https://drive.google.com/uc\?id\=1Rp_mykKNtBYURqZsEcIWGzPqXbI3NZwG
from google.colab import files
from io import BytesIO
uploaded = files.upload()


/content
/content/input_audio
Downloading...
From: https://drive.google.com/uc?id=1Rp_mykKNtBYURqZsEcIWGzPqXbI3NZwG
To: /content/input_audio/8888.mp3
100% 110k/110k [00:00<00:00, 98.5MB/s]


In [9]:
#@title 五、上传图片
# 创建相关的文件夹
# 如果只是运行demo,点击取消上传即可
%cd /content
!rm -rf input_image
!mkdir input_image
%cd /content/input_image
!gdown https://drive.google.com/uc\?id\=1skZ4e0ESbrRcUsfUXgaG1LR4TlBeGIkS
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content
/content/input_image
Downloading...
From: https://drive.google.com/uc?id=1skZ4e0ESbrRcUsfUXgaG1LR4TlBeGIkS
To: /content/input_image/8888.jpg
100% 76.6k/76.6k [00:00<00:00, 85.3MB/s]


In [7]:
#@title 五、上传驱动视频
# 创建相关的文件夹
# 如果只是运行demo,点击取消上传即可
%cd /content
!rm -rf temp_video
!mkdir temp_video
!rm -rf input_video
!mkdir input_video
%cd /content/temp_video
# 先下载一个模板视频，用户可以自己上传，也可以直接用我给的这个模板视频进行动作迁移
!gdown https://drive.google.com/uc\?id\=17nbM-ccJ5mo4_QZyxGCXnZsr8689oj7N

from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content
/content/temp_video
Downloading...
From: https://drive.google.com/uc?id=17nbM-ccJ5mo4_QZyxGCXnZsr8689oj7N
To: /content/temp_video/8888.mp4
100% 3.70M/3.70M [00:00<00:00, 218MB/s]


In [15]:
#@title 六、将图片进行动作迁移
%cd /content/Thin-Plate-Spline-Motion-Model
!python demo.py --config config/vox-256.yaml --checkpoint checkpoints/vox.pth.tar --source_image /content/input_image/8888.jpg --driving_video /content/temp_video/8888.mp4

/content/Thin-Plate-Spline-Motion-Model
/content/Thin-Plate-Spline-Motion-Model/demo.py:147: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  source_image = imageio.imread(opt.source_image)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an

In [16]:
# 将生成的视频移动到指定位置
!mv /content/Thin-Plate-Spline-Motion-Model/result.mp4  /content/input_video

# 放大图片"scale=640:820"这个参数可以自己设置,一般需要根据输入的图片和视频尺寸来确定,也可以自己调整
!ffmpeg -i /content/input_video/result.mp4 -vf scale=640:820 -y /content/input_video/input.mp4

# 删除临时视频
!rm -rf /content/input_video/result.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [18]:
#@title 七、用wav2lip篡改嘴型
# checkpoints 可以用wav2lip.pth 或者wav2lip_gan.pt
# 建议调小batch size, 调成8
%cd /content/Wav2Lip
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/input_video/input.mp4' --audio '/content/input_audio/8888.mp3'

/content/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 205
Extracting raw audio...
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsn

In [19]:
# 将Wav2Lip生成的视频转化成图片保存到images
%cd /content
!mkdir images
%cd /content/Wav2Lip
import cv2
from tqdm import tqdm
from os import path
import os
vidcap = cv2.VideoCapture("/content/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/images', str(frameNumber).zfill(5)+'.jpg'), image)

/content
/content/Wav2Lip
FPS:  30.0 Frames:  203


100%|██████████| 203/203 [00:02<00:00, 93.56it/s]


In [20]:
#@title 八、用CodeFormer进行人脸修复
# 用CodeFormer进行推理
%cd /content/CodeFormer
## 只修复人脸
# CODEFORMER_FIDELITY = 0.5
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images
# 整体图片修复
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/images --bg_upsampler realesrgan

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:01<00:00, 37.9MB/s]
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth" to /content/CodeFormer/weights/CodeFormer/codeformer.pth

100% 359M/359M [00:01<00:00, 325MB/s]
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 366MB/s] 
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth" to /content/CodeFormer/weights/facelib/parsing_parsenet.pth

100% 81.4M/81.4M [00:00<00:00, 174MB/s]
[1/203] Processing: 00000.jpg
	detect 1 faces
[2/203] Processing: 00001.jpg
	det

In [21]:
# 生成修复后的新视频
# 提取视频中的音频，用于后续处理
!ffmpeg -i /content/Wav2Lip/results/result_voice.mp4 /content/output/final.mp3
# 合并成高清视频
# ffmpeg 将文件中每一帧图片合并成mp4
%cd /content/CodeFormer/results/images_0.7/final_results
!ffmpeg -framerate 30 -i %05d.png  -pix_fmt yuv420p  /content/CodeFormer/results/images_0.7/high_definition.mp4
%cd /content/
# 音视频合成
!ffmpeg -i /content/CodeFormer/results/images_0.7/high_definition.mp4 -i /content/output/final.mp3 -c copy /content/output/final.mp4
# 删除临时音频
!rm -rf /content/output/final.mp3
print('恭喜您高清数字人视频合并成功,请查看路径/content/output/final.mp4')
print('恭喜您高清数字人视频合并成功,请查看路径/content/output/final.mp4')
print('恭喜您高清数字人视频合并成功,请查看路径/content/output/final.mp4')


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e